# Automation Workflow Tutorial <a class="tocSkip">

This notebook contains an interactive introduction to the Python DapticsClient class,
a simplified interface for accessing the Daptics GraphQL API for the optimization of
experimental design.

Documentation for using the DapticsClient class (implemented in the daptics_client.py
file in this folder) is included as comment lines in the interactive Python cells of
this notebook.

For additional help or information, please visit or contact Daptics.

On the web at https://daptics.ai
By email at support@daptics.ai

Daptics API Version 0.10.0
Copyright (c) 2020 Daptics Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), the rights to use, copy, modify, merge, publish, and/or distribute, copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

You do not have the right to sub-license or sell copies of the Software.

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Installation and Setup<a class="tocSkip">

Before running this project, please make sure that your Jupyter Python environment supports Python 3, and has these required packages installed:
*   chardet
*   urllib3
*   requests
*   gql

You will also need a validated user account on the Daptics API server.  You can create an account by [registering](https:daptics.ai/register) at https://daptics.ai, or contacting our sales department at sales@daptics.ai

See the **01_README.ipynb** notebook in this folder for more information on installing necessary modules
to work with the Daptics API.

Review the **02_Terminology.ipynb** notebook to gain an understanding of how Daptics works, and
way you set up the engine using experimental space parameters.

The next cell gives an example of the information necessary to connect, log in, create a session,
and define the "experimental space" for your campaign.

In [ ]:
# Set up the API client environment.
# Requirements are Python 3, and the `gql` and `requests` libraries.

# Import required classes from the daptics_client module.
from daptics_client import DapticsClient, DapticsTaskType, DapticsExperimentsType
from datetime import datetime
import json
import os.path

# Initialize values for all the variables you will use for this example.

# The URL for the beta API server.
api_host_url = 'https://beta-api.daptics.ai'

# Configuration options used for automated processing.
config = {
    # Create CSV files at each step in this directory.
    'auto_export_path': output_path,
    # Let long-running tasks execute for up to one hour.
    'auto_task_timeout': 3600,
    # Automatically generate the next design when results are uploaded.
    'auto_generate_next_design': True
}

# The credentials for an active Daptics account.
# Please contact sales@daptics.ai for information on
# how to obtain an account.
email = 'joe@daptics.ai'
password = 'joes-password'

# Make a unique name for your session.
timestamp_string = datetime.now().strftime('%Y%m%d%H%M%S')
session_name = 'Automated Example {}'.format(timestamp_string)
# A description for the session
session_description = 'Fully automated example campaign'
# You will store the session unique identifier and the generation
# number for the last design in a file that your automation workflow
# software can read. The values in this file will let the automation
# software pick up the appropriate design file.
session_file = './output/session.json'

# The location where Daptics will create CSV files for designed
# experiments.
output_path = './output'

# The location of the CSV file that your automation workflow software
# will create after performing all the experiments in a Daptics-
# generated design and adding the Response value for each
# experiment.
csv_experiments_file = './input/experiments.csv'

# The location of the CSV file that defines the names and possible
# values for each input parameter for an individual experiment.
csv_space_file = './input/experimental_space.csv'

# The experimental meta-parameters that define the type of experiment
# campaign you will be doing, and the number of experiments that will
# be explored at each gneration. The combination of these meta-
# parameters and the definition of the input parameters is called the
# "experimental space" for the campaign.
space_params = {
    # The 'factorial' experimental space type is used for
    # unconstrained input parameters.
    'space': { 'type': 'factorial' },
    # Each design generated will contain 30 independent experiments.
    'populationSize': 30,
    # And will have 2 copies of these experiments, so the total
    # number of experiments to be performed at each generation will
    # be 30 * 3 = 90.
    'replicates': 2
}

## Generating the First Design <a class="tocSkip">

With modules and variables set up, you must place a CSV file that
defines the names and possible values for each input parameter
that will be used in your experiemnts. This should be saved to a
CSV file at the location specified in the `csv_space_file` variable.

Once this file is there, you can connect to the API server, log in,
create a session for your experimental campaign, and have Daptics
generate a first batch of experiments for the automation workflow
software to perform and assay.

In [ ]:
# With all the Python requirements satisfied, and using the variables
# previously defined, you can go ahead and generate your first
# experimental design generation.

# Create a Python client instance, connecting to the beta API server.
daptics = DapticsClient(api_host_url)

# Set up the options that will fully automate client processing.
daptics.options = config

# The 'connect' method will connect to the API server and obtain the
# GraphQL schema.
daptics.connect()

# Log into the API using your Daptics account credentials.
daptics.login(email, password)

# Create a new 'session' for this experimental campaign.
daptics.create_session(session_name, session_description)

# Together with these meta-parameters, upload the names and values of
# the experimental input parameters to completely initialize the
# Daptics engine for this campaign.
daptics.put_experimental_parameters_csv(csv_space_file, metaparams)

# Now the Daptics engine is set up. At this point, if you have any
# initial experiments that have been performed by your workflow,
# instead of calling `daptics.generate_design()`, you can upload
# these experiments from a CSV that your automation workflow software
# has created, containing the input parameter and the response values
# for each of these "initial experiments".

# Here is how you would be use the API to upload initial experiments:
# daptics.put_experiments_csv(
#    DapticsExperimentsType.INITIAL_EXTRAS_ONLY,
#    csv_experiments_file)

# If you have no initial experiments to upload, you will just
# ask Daptics to generate the first design.
daptics.generate_design()

# Because the `auto_generate_next_design` and `auto_export_path`
# options were set in the client, if either the `put_experiments_csv`
# or `generate_design` methods were called, the Python script will
# block here until the design for the first generation has completed,
# at which point the design will have been saved into the file
# `./output/auto_gen1_design.csv`.

# Save the session id, generation number and design file location,
# so that the automation software can read the design and reconnect
# to the session to upload experimental results and generate the
# next design.

# The `auto_gen1_design.csv` file and design files for subsequent
# generations will be created automatically using the format shown
# here, since the `auto_export_path` option was set.
session_id = daptics.session_id
gen = daptics.gen
csv_design_file = os.path.join(
    output_path, 'auto_gen{}_design.csv'.format(gen))
session_data = {
    'session_id': daptics.session_id,
    'gen': gen,
    'csv_design_file': csv_design_file
}
with open(session_file, 'wt') as f:
    json.dump(session_data, f)

## In The Loop <a class="tocSkip">

After the initial design is generated, the automation workflow software
will read the input parameters from the file `auto_gen1_design.csv`
in the directory specified by the Python variable `output_path`.
Each row in the CSV file represents a unique experiment, with the
values specified for each input parameter in the column with the
parameter's name.

The workflow software must convert these input parameter names and
values into an experimental workflow, mapping inputs to plate wells,
pipetting appropriate reagents, performing incubation and
filtration, etc., and finally conducting assays.
The workflow software must also convert the assay results
from each experiment into a single target number, called the
experiment "Response". This is the number that the Daptics AI will optimize.

The workflow software must add these responses to the Daptics-generated
design file. The response to each experiment should be written into the `Response`
column of the CSV file in the row that corresponds to the experiment.
The CSV file is saved to the location specified in the `csv_experiments_file`
Python variable: `./input/experiments.csv`.

Now the workflow software can execute the next part of the Daptics process,
simply uploading the experiments and their responses and generating the
next design. To do so you must reconnect to the session using the
`session_id` value saved previously. Once reconnected, the only
API call necessary is to upload the experiments:

In [ ]:
# If you put this code into a separate script for your automation
# workflow engine, make sure to include code from the first Python
# cell in this tutorial that contains the imports and variables used
# here.

# To begin you reload the session data you saved when you generated
# the previous design.
with open(session_file, 'rt') as f:
    session_data = json.load(f)

# You follow the same connect and login procedure as for the first
# design, then reconnect to your session.

# Create a Python client instance, connecting to the beta API server.
daptics = DapticsClient(api_host_url)

# Set up the options that will fully automate client processing.
daptics.options = config

# The 'connect' method will connect to the API server and obtain the
# GraphQL schema.
daptics.connect()

# Log into the API using your Daptics account credentials.
daptics.login(email, password)

# Reconnect to the session after logging in. Here you rely
# on the 'session_id' value saved in the session file.
daptics.reconnect_session(session_data['session_id'])

# Upload the experiments file that your automation workflow software
# created, containing the previous design and the response values
# for each experiment in the design.
daptics.put_experiments_csv(
    DapticsExperimentsType.DESIGNED_WITH_OPTIONAL_EXTRAS,
    csv_experiments_file)

# Because the `auto_generate_next_design` and `auto_export_path`
# options were set in the client, the Python script will block here
# until the design for the next generation has completed, at which
# point the design will have been saved into the file
# `./output/auto_genN_design.csv`, where N is the generation number,
# ready for your workflow software to read.

# Before you exit the script, you update the information in the
# session file, for use by the workfow software.
session_id = daptics.session_id
gen = daptics.gen
csv_design_file = os.path.join(
    output_path, 'auto_gen{}_design.csv'.format(gen))
session_data = {
    'session_id': daptics.session_id,
    'gen': gen,
    'csv_design_file': csv_design_file
}
with open(session_file, 'wt') as f:
    json.dump(session_data, f)

# Now the workflow software can read the design file, execute more
# experiments, and continue the loop. You can exit the script now.

# Or you can optionally generate analytics graphs at each step of
# the loop. These PDF files can then be picked up by automation
# software and included in your lab notebook, etc.

# Generate any analytics files that are available for this generation.
# Since the `auto_task_timeout` option has been set, the script will
# block until the files are ready to be downloaded.
daptics.generate_analytics()

# Fetch the PDF analytics files via authenticated HTTP, and save them
# to the './output' directory, where your automation workflow
# software can pick them up.
daptics.get_all_analytics_files(output_path)